In [5]:
import random
import simpy

## Модель банка с отказами

### Инициализация стартовой модели

In [6]:
RANDOM_SEED = 42
NEW_CUSTOMERS = 5  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 3  # Max. customer patience

In [7]:
def source(env, number, interval, counter):
    """Source generates customers randomly"""
    for i in range(number):
        c = customer(env, 'Customer%02d' % i, counter, time_in_bank=12.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)

In [8]:
def customer(env, name, counter, time_in_bank):
    """Customer arrives, is served and leaves."""
    arrive = env.now
    print('%7.4f %s: Here I am' % (arrive, name))

    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Wait for the counter or abort at the end of our tether
        results = yield req | env.timeout(patience)

        wait = env.now - arrive

        if req in results:
            # We got to the counter
            print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))

            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print('%7.4f %s: Finished' % (env.now, name))

        else:
            # We reneged
            print('%7.4f %s: RENEGED after %6.3f' % (env.now, name, wait))

In [9]:
# Setup and start the simulation
print('Bank renege')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Start processes and run
counter = simpy.Resource(env, capacity=1)
env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter))
env.run()

Bank renege
 0.0000 Customer00: Here I am
 0.0000 Customer00: Waited  0.000
 3.8595 Customer00: Finished
10.2006 Customer01: Here I am
10.2006 Customer01: Waited  0.000
12.7265 Customer02: Here I am
13.9003 Customer02: RENEGED after  1.174
23.7507 Customer01: Finished
34.9993 Customer03: Here I am
34.9993 Customer03: Waited  0.000
37.9599 Customer03: Finished
40.4798 Customer04: Here I am
40.4798 Customer04: Waited  0.000
43.1401 Customer04: Finished


### В этом примере моделируется окно обслуживания банка и клиенты, прибывающие в случайное время. У каждого клиента есть ограниченное время в очереди (отказы). Обслуживание длится некоторое время.

### Задание (выполнять в отдельном .py и .ipynb файлах)

Реализовать дополнительно следующую логику:

1. Сделать модель ограниченной по времени, а не по количеству клиентов. Клиентов сделать с двумя разными типами задач (отношение $3:1$). Задача первого типа с параметром $time\_in\_bank=9.0$, второго - $time\_in\_bank=15.0$.
2. Количество окон обслуживания сделать параметром модели. Приоритетов и выталкивания в модели нет.
3. Ввести экономическую функцию: за каждого обслуженно клиента с задачей первого типа прибыль $3$, второго типа - $6$, за потерянного клиента первого типа издержки $1.1$, второго типа - $2.5$, фиксированные издержки за сотрудников составляют $0.03$ в единицу времени на человека, а также бонус за задачу первого типа $0.3$, второго типа - $0.5$.

Максимизировать чистую прибыль по параметру числа окон на горизонте 5000 временых шагов.